# Most efficient cultural holiday in Florence - Part 2

### by Francesca Scalese

**NOTE:** This is the second part of the week 4 final assignment. Only data used in this project will be talked about. The full project will be published in the week 5 assignment

## 2. Data acquisition and cleaning

### 2.1 Data source

The source of data about the location of museums, churches and hotels is the website [Foursquare](https://foursquare.com/).

### 2.2 Importing, cleaning data and feature selection

 **2.2.1 Import needed libraries**

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 8.6 MB/s  eta 0:00:01
  Created wheel for folium: file

**2.2.2 Define Foursquare Credentials and Version**

In order to use Foursquare to acquiring data, it is needed to define credentials  and versions:

In [2]:
CLIENT_ID = 'Y2E14U4GP44DD3R0E1MLC51OSZ33I0QUJ1RTVXFGM2Y4NZBB' 
CLIENT_SECRET = '3LJYB5GP2QSSJ04W5F2ZGWOSVPOIVRN0FUMQ1IQYORDZ2OD4' 
ACCESS_TOKEN = 'SBC5N30W3PLCPW5RNKMJBRM3HYTAZRO4LMUHMYIARTS13MEM' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y2E14U4GP44DD3R0E1MLC51OSZ33I0QUJ1RTVXFGM2Y4NZBB
CLIENT_SECRET:3LJYB5GP2QSSJ04W5F2ZGWOSVPOIVRN0FUMQ1IQYORDZ2OD4


Florence is the city the client is interested in. Therefore, it will be our starting point. Its coordinates will be defined using geocode. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent foursquare_agent, as shown below.

**Note:** *Firenze, Toscana* is the italian name for *Florence, Tuscany*

In [3]:
address = 'Firenze, Toscana'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7698712 11.2555757


**2.2.3 Searching museums and creating a dataframe**

Now define a query to search museums in Florence(in italian *museo* means museum).

In [4]:
search_query_museum = 'Museo'
radius = 500
print(search_query_museum + ' .... OK!')

Museo .... OK!


Define the corresponding URL

In [5]:
url_museum = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_museum, radius, LIMIT)
url_museum

'https://api.foursquare.com/v2/venues/search?client_id=Y2E14U4GP44DD3R0E1MLC51OSZ33I0QUJ1RTVXFGM2Y4NZBB&client_secret=3LJYB5GP2QSSJ04W5F2ZGWOSVPOIVRN0FUMQ1IQYORDZ2OD4&ll=43.7698712,11.2555757&oauth_token=SBC5N30W3PLCPW5RNKMJBRM3HYTAZRO4LMUHMYIARTS13MEM&v=20180604&query=Museo&radius=500&limit=30'

Send the GET Request and examine the results

In [6]:
results_museum = requests.get(url_museum).json()
results_museum

{'meta': {'code': 200, 'requestId': '60afa17fead31a78280291d1'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4e8026c577c8c61e00ca31c5',
    'name': 'Gucci Museo',
    'location': {'address': 'Piazza Della Signoria, 10',
     'lat': 43.76976229374217,
     'lng': 11.256747341104528,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.76976229374217,
       'lng': 11.256747341104528}],
     'distance': 94,
     'postalCode': '50122',
     'cc': 'IT',
     'city': 'Firenze',
     'state': 'Toscana',
     'country': 'Italia',
     'formattedAddress': ['Piazza Della Signoria, 10',
      '50122 Firenze Toscana',
      'Italia']},
    'categories': [{'id': '4bf58dd8d48988d181941735',
      'name': 'Museum',
      'pluralName': 'Museums',
      'shortName': 'Museum',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_',
       'suffix': '.png'},
      'primary': True}],
    'ref

Get relevant part of JSON and transform it into a pandas dataframe

In [7]:
# assign relevant part of JSON to venues
venues_museum = results_museum['response']['venues']

# tranform venues into a dataframe
df_museum = json_normalize(venues_museum)
df_museum.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,4e8026c577c8c61e00ca31c5,Gucci Museo,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",v-1622122879,False,"Piazza Della Signoria, 10",43.769762,11.256747,"[{'label': 'display', 'lat': 43.76976229374217...",94,50122,IT,Firenze,Toscana,Italia,"[Piazza Della Signoria, 10, 50122 Firenze Tosc...",NaN,NaN
1,4bd00aa8b221c9b6c80cd3d0,Museo dell'Opera del Duomo,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",v-1622122879,False,Via della Canonica 1,43.772988,11.257492,"[{'label': 'display', 'lat': 43.77298793077224...",379,50122,IT,Firenze,Toscana,Italia,"[Via della Canonica 1 (Via dei Servi), 50122 F...",Via dei Servi,NaN
2,53524fb4498eb5c71458096a,Museo di Palazzo Vecchio (Museo Civico),"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",v-1622122879,False,Piazza della Signoria,43.769422,11.256502,"[{'label': 'display', 'lat': 43.7694221597928,...",89,NaN,IT,Firenze,Toscana,Italia,"[Piazza della Signoria, Firenze Toscana, Italia]",NaN,NaN
3,4bd01a269854d13a1620f74d,Museo Galileo - Istituto e Museo di Storia del...,"[{'id': '4bf58dd8d48988d191941735', 'name': 'S...",v-1622122879,False,Piazza dei Giudici,43.767799,11.256202,"[{'label': 'display', 'lat': 43.76779899076643...",236,50122,IT,Firenze,Toscana,Italia,"[Piazza dei Giudici, 50122 Firenze Toscana, It...",NaN,NaN
4,4b49d545f964a520507426e3,Museo Nazionale del Bargello,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",v-1622122879,False,"Via del Proconsolo, 4",43.770058,11.258340,"[{'label': 'display', 'lat': 43.77005835195692...",223,50122,IT,Firenze,Toscana,Italia,"[Via del Proconsolo, 4, 50122 Firenze Toscana,...",NaN,NaN


Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_museum = ['name', 'categories'] + [col for col in df_museum.columns if col.startswith('location.')] + ['id']
df_museum_filtered = df_museum.loc[:, filtered_columns_museum]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_museum_filtered['categories'] = df_museum_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_museum_filtered.columns = [column.split('.')[-1] for column in df_museum_filtered.columns]

df_museum_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Gucci Museo,Museum,"Piazza Della Signoria, 10",43.769762,11.256747,"[{'label': 'display', 'lat': 43.76976229374217...",94,50122,IT,Firenze,Toscana,Italia,"[Piazza Della Signoria, 10, 50122 Firenze Tosc...",NaN,NaN,4e8026c577c8c61e00ca31c5
1,Museo dell'Opera del Duomo,Museum,Via della Canonica 1,43.772988,11.257492,"[{'label': 'display', 'lat': 43.77298793077224...",379,50122,IT,Firenze,Toscana,Italia,"[Via della Canonica 1 (Via dei Servi), 50122 F...",Via dei Servi,NaN,4bd00aa8b221c9b6c80cd3d0
2,Museo di Palazzo Vecchio (Museo Civico),Art Museum,Piazza della Signoria,43.769422,11.256502,"[{'label': 'display', 'lat': 43.7694221597928,...",89,NaN,IT,Firenze,Toscana,Italia,"[Piazza della Signoria, Firenze Toscana, Italia]",NaN,NaN,53524fb4498eb5c71458096a
3,Museo Galileo - Istituto e Museo di Storia del...,Science Museum,Piazza dei Giudici,43.767799,11.256202,"[{'label': 'display', 'lat': 43.76779899076643...",236,50122,IT,Firenze,Toscana,Italia,"[Piazza dei Giudici, 50122 Firenze Toscana, It...",NaN,NaN,4bd01a269854d13a1620f74d
4,Museo Nazionale del Bargello,Art Museum,"Via del Proconsolo, 4",43.770058,11.258340,"[{'label': 'display', 'lat': 43.77005835195692...",223,50122,IT,Firenze,Toscana,Italia,"[Via del Proconsolo, 4, 50122 Firenze Toscana,...",NaN,NaN,4b49d545f964a520507426e3


Now some column will be eliminated, mantaining only name, category, address, latitude and longitude.

In [9]:
df_museum_filtered = df_museum_filtered[['name','categories', 'address','lat','lng']]
df_museum_filtered.head()

,name,categories,address,lat,lng
0,Gucci Museo,Museum,"Piazza Della Signoria, 10",43.769762,11.256747
1,Museo dell'Opera del Duomo,Museum,Via della Canonica 1,43.772988,11.257492
2,Museo di Palazzo Vecchio (Museo Civico),Art Museum,Piazza della Signoria,43.769422,11.256502
3,Museo Galileo - Istituto e Museo di Storia del...,Science Museum,Piazza dei Giudici,43.767799,11.256202
4,Museo Nazionale del Bargello,Art Museum,"Via del Proconsolo, 4",43.770058,11.258340


Let's have a look to the categories.

In [10]:
df_museum_filtered['categories'].value_counts()

Art Museum               8
Museum                   7
Art Gallery              4
History Museum           3
Science Museum           2
City Hall                1
Scenic Lookout           1
Bookstore                1
Café                     1
Library                  1
General Entertainment    1
Name: categories, dtype: int64

Some categories do not belong to museums but they will be kept in the dataframe because they may have some historical interest or are connected to cultural sites.

Visualize the museums on the map

In [41]:
venues_map_museum = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred in Florence

# the blue circles represent Florence museums
for lat, lng, label in zip(df_museum_filtered.lat, df_museum_filtered.lng, df_museum_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_museum)

# display map
venues_map_museum

**2.2.4 Searching churches and creating dataframe**

Now the previous procedure seen for the researc about museums will be applied on churches.

First of all, define a query to search churches (in italian *chiesa* means church).

In [12]:
search_query_church = 'Chiesa'
radius = 500
print(search_query_church + ' .... OK!')

Chiesa .... OK!


Define the corresponding URL

In [13]:
url_church = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_church, radius, LIMIT)
url_church

'https://api.foursquare.com/v2/venues/search?client_id=Y2E14U4GP44DD3R0E1MLC51OSZ33I0QUJ1RTVXFGM2Y4NZBB&client_secret=3LJYB5GP2QSSJ04W5F2ZGWOSVPOIVRN0FUMQ1IQYORDZ2OD4&ll=43.7698712,11.2555757&oauth_token=SBC5N30W3PLCPW5RNKMJBRM3HYTAZRO4LMUHMYIARTS13MEM&v=20180604&query=Chiesa&radius=500&limit=30'

Send the GET result and examine the resutls

In [14]:
results = requests.get(url_church).json()
results

{'meta': {'code': 200, 'requestId': '60afa182d4a27e42bbd18f7e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '51221e06e4b0b9b410cbdf62',
    'name': 'Chiesa di San Carlo dei Lombardi',
    'location': {'lat': 43.77066521192117,
     'lng': 11.255226315740265,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.77066521192117,
       'lng': 11.255226315740265}],
     'distance': 92,
     'cc': 'IT',
     'city': 'Firenze',
     'state': 'Toscana',
     'country': 'Italia',
     'formattedAddress': ['Firenze Toscana', 'Italia']},
    'categories': [{'id': '4bf58dd8d48988d132941735',
      'name': 'Church',
      'pluralName': 'Churches',
      'shortName': 'Church',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/religious_church_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1622122882',
    'hasPerk': False},
   {'id': '4c9f024154c8a1cd6678934b',
    'name':

Get relevant part of JSON and transform it into a pandas dataframe

In [15]:
# assign relevant part of JSON to venues
venues_church = results['response']['venues']

# tranform venues into a dataframe
df_church = json_normalize(venues_church)
df_church.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.neighborhood,location.crossStreet
0,51221e06e4b0b9b410cbdf62,Chiesa di San Carlo dei Lombardi,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1622122882,False,43.770665,11.255226,"[{'label': 'display', 'lat': 43.77066521192117...",92,IT,Firenze,Toscana,Italia,"[Firenze Toscana, Italia]",NaN,NaN,NaN,NaN
1,4c9f024154c8a1cd6678934b,Chiesa di Santa Felicita,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1622122882,False,43.766968,11.252400,"[{'label': 'display', 'lat': 43.76696765452083...",411,IT,Firenze,Toscana,Italia,"[Piazza di Santa Felicita, 50125 Firenze Tosca...",Piazza di Santa Felicita,50125,NaN,NaN
2,4bc6fd362f94d13a7a0c117f,Chiesa di Orsanmichele,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1622122882,False,43.770671,11.255205,"[{'label': 'display', 'lat': 43.7706709023181,...",93,IT,Firenze,Toscana,Italia,"[Via dei Calzaiuoli, 50127 Firenze Toscana, It...",Via dei Calzaiuoli,50127,NaN,NaN
3,4c629b7fec94a593479a29ca,Chiesa di Dante,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1622122882,False,43.771241,11.257141,"[{'label': 'display', 'lat': 43.77124068518998...",197,IT,Firenze,Toscana,Italia,"[Via Santa Margherita, 50123 Firenze Toscana, ...",Via Santa Margherita,50123,NaN,NaN
4,4c62a105e1621b8d83d52353,Orzanmichele Chiesa,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",v-1622122882,False,43.769931,11.254812,"[{'label': 'display', 'lat': 43.76993130530206...",61,IT,Firenze,Toscana,Italia,"[Via dei Calzaioli, 14-red-32-red, 50122 Firen...","Via dei Calzaioli, 14-red-32-red",50122,NaN,NaN


Define information of interest and filter dataframe

In [16]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_church = ['name', 'categories'] + [col for col in df_church.columns if col.startswith('location.')] + ['id']
df_church_filtered = df_church.loc[:, filtered_columns_church]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_church_filtered['categories'] = df_church_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_church_filtered.columns = [column.split('.')[-1] for column in df_church_filtered.columns]

df_church_filtered.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,neighborhood,crossStreet,id
0,Chiesa di San Carlo dei Lombardi,Church,43.770665,11.255226,"[{'label': 'display', 'lat': 43.77066521192117...",92,IT,Firenze,Toscana,Italia,"[Firenze Toscana, Italia]",NaN,NaN,NaN,NaN,51221e06e4b0b9b410cbdf62
1,Chiesa di Santa Felicita,Church,43.766968,11.252400,"[{'label': 'display', 'lat': 43.76696765452083...",411,IT,Firenze,Toscana,Italia,"[Piazza di Santa Felicita, 50125 Firenze Tosca...",Piazza di Santa Felicita,50125,NaN,NaN,4c9f024154c8a1cd6678934b
2,Chiesa di Orsanmichele,Church,43.770671,11.255205,"[{'label': 'display', 'lat': 43.7706709023181,...",93,IT,Firenze,Toscana,Italia,"[Via dei Calzaiuoli, 50127 Firenze Toscana, It...",Via dei Calzaiuoli,50127,NaN,NaN,4bc6fd362f94d13a7a0c117f
3,Chiesa di Dante,Church,43.771241,11.257141,"[{'label': 'display', 'lat': 43.77124068518998...",197,IT,Firenze,Toscana,Italia,"[Via Santa Margherita, 50123 Firenze Toscana, ...",Via Santa Margherita,50123,NaN,NaN,4c629b7fec94a593479a29ca
4,Orzanmichele Chiesa,Museum,43.769931,11.254812,"[{'label': 'display', 'lat': 43.76993130530206...",61,IT,Firenze,Toscana,Italia,"[Via dei Calzaioli, 14-red-32-red, 50122 Firen...","Via dei Calzaioli, 14-red-32-red",50122,NaN,NaN,4c62a105e1621b8d83d52353


Now some column will be eliminated, mantaining only name, category, address, latitude and longitude.

In [17]:
df_church_filtered = df_church_filtered[['name','categories', 'address','lat','lng']]
df_church_filtered.head()

,name,categories,address,lat,lng
0,Chiesa di San Carlo dei Lombardi,Church,NaN,43.770665,11.255226
1,Chiesa di Santa Felicita,Church,Piazza di Santa Felicita,43.766968,11.252400
2,Chiesa di Orsanmichele,Church,Via dei Calzaiuoli,43.770671,11.255205
3,Chiesa di Dante,Church,Via Santa Margherita,43.771241,11.257141
4,Orzanmichele Chiesa,Museum,"Via dei Calzaioli, 14-red-32-red",43.769931,11.254812


Let's have a look on the categories

In [18]:
df_church_filtered['categories'].value_counts()

Church    25
Museum     1
Name: categories, dtype: int64

Visualize the position of the curches on the map

In [42]:
venues_map_church = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred in Florence

# blue circles represent curches in Florence
for lat, lng, label in zip(df_church_filtered.lat, df_church_filtered.lng, df_church_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_church)

# display map
venues_map_church

**2.2.5 Unite churches and museums in a unique dataframe**

Let's have a look about the shape of the previous dataframe

In [21]:
print(df_museum_filtered.shape)
print(df_church_filtered.shape)

(30, 5)
(26, 5)


Now unite the dataframes about churches and museums

In [22]:
df_total = df_museum_filtered.append(df_church_filtered, ignore_index = True)
df_total.head()

,name,categories,address,lat,lng
0,Gucci Museo,Museum,"Piazza Della Signoria, 10",43.769762,11.256747
1,Museo dell'Opera del Duomo,Museum,Via della Canonica 1,43.772988,11.257492
2,Museo di Palazzo Vecchio (Museo Civico),Art Museum,Piazza della Signoria,43.769422,11.256502
3,Museo Galileo - Istituto e Museo di Storia del...,Science Museum,Piazza dei Giudici,43.767799,11.256202
4,Museo Nazionale del Bargello,Art Museum,"Via del Proconsolo, 4",43.770058,11.258340


In [23]:
df_total.shape

(56, 5)

The shape of the final data frame correspond to the sum of the previous ones.

Now let's visualize churches and museums on a map

In [43]:
venues_map_total = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred in Florence

# the blue circles represent Florence museums and churches
for lat, lng, label in zip(df_total.lat, df_total.lng, df_total.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_total)

# display map
venues_map_total

**2.2.6 Searching hotels**

Now define a query to search hotels in Florence.


In [25]:
search_query_hotel = 'Hotel'
radius = 500
print(search_query_hotel + ' .... OK!')

Hotel .... OK!


Define the corresponding URL

In [26]:
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query_hotel, radius, LIMIT)
url_hotel

'https://api.foursquare.com/v2/venues/search?client_id=Y2E14U4GP44DD3R0E1MLC51OSZ33I0QUJ1RTVXFGM2Y4NZBB&client_secret=3LJYB5GP2QSSJ04W5F2ZGWOSVPOIVRN0FUMQ1IQYORDZ2OD4&ll=43.7698712,11.2555757&oauth_token=SBC5N30W3PLCPW5RNKMJBRM3HYTAZRO4LMUHMYIARTS13MEM&v=20180604&query=Hotel&radius=500&limit=30'

Send the GET request and examine the results

In [27]:
results_hotel = requests.get(url_hotel).json()
results_hotel

{'meta': {'code': 200, 'requestId': '60afa1942b4ee825331799d4'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '5b5cb0e8d03360002cac6465',
    'name': 'Hotel Savoy Firenze',
    'location': {'address': 'Piazza Della Repubblica 7',
     'lat': 43.77188419874087,
     'lng': 11.254516474382802,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.77188419874087,
       'lng': 11.254516474382802}],
     'distance': 239,
     'postalCode': '50123',
     'cc': 'IT',
     'city': 'Firenze',
     'state': 'Toscana',
     'country': 'Italia',
     'formattedAddress': ['Piazza Della Repubblica 7',
      '50123 Firenze Toscana',
      'Italia']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId

Get relevant part of JSON and transform it into a pandas dataframe

In [28]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']

# tranform venues into a dataframe
df_hotel = json_normalize(venues_hotel)
df_hotel.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,5b5cb0e8d03360002cac6465,Hotel Savoy Firenze,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1622122901,False,Piazza Della Repubblica 7,43.771884,11.254516,"[{'label': 'display', 'lat': 43.77188419874087...",239,50123,IT,Firenze,Toscana,Italia,"[Piazza Della Repubblica 7, 50123 Firenze Tosc...",NaN,NaN
1,4c15708177cea5932643d260,Relais Hotel Uffizi,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1622122901,False,Via del buco 16,43.769115,11.254923,"[{'label': 'display', 'lat': 43.76911484178026...",99,NaN,IT,Firenze,Toscana,Italia,"[Via del buco 16, Firenze Toscana, Italia]",NaN,NaN
2,4bc8848fdc55eee14a85e8ac,Hotel Bernini Palace,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1622122901,False,"Piazza di San Firenze, 29",43.769440,11.257540,"[{'label': 'display', 'lat': 43.76944, 'lng': ...",165,50100,IT,Firenze,Toscana,Italia,"[Piazza di San Firenze, 29 (Borgo de' Greci), ...",Borgo de' Greci,NaN
3,5cd89186db1d81002cb6e9d8,Il Gattopardo Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1622122901,False,NaN,43.770559,11.256459,"[{'label': 'display', 'lat': 43.770559, 'lng':...",104,50122,IT,Firenze,Toscana,Italia,"[50122 Firenze Toscana, Italia]",NaN,NaN
4,4d95ac749079b1f77604e609,Hotel Calzaiuoli,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1622122901,False,NaN,43.771324,11.255368,"[{'label': 'display', 'lat': 43.77132414522816...",162,NaN,IT,NaN,NaN,Italia,[Italia],NaN,NaN


Define information of interest and filter dataframe


In [29]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_hotel = ['name', 'categories'] + [col for col in df_hotel.columns if col.startswith('location.')] + ['id']
df_hotel_filtered = df_hotel.loc[:, filtered_columns_hotel]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_hotel_filtered['categories'] = df_hotel_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_hotel_filtered.columns = [column.split('.')[-1] for column in df_hotel_filtered.columns]

df_hotel_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Hotel Savoy Firenze,Hotel,Piazza Della Repubblica 7,43.771884,11.254516,"[{'label': 'display', 'lat': 43.77188419874087...",239,50123,IT,Firenze,Toscana,Italia,"[Piazza Della Repubblica 7, 50123 Firenze Tosc...",NaN,5b5cb0e8d03360002cac6465
1,Relais Hotel Uffizi,Hotel,Via del buco 16,43.769115,11.254923,"[{'label': 'display', 'lat': 43.76911484178026...",99,NaN,IT,Firenze,Toscana,Italia,"[Via del buco 16, Firenze Toscana, Italia]",NaN,4c15708177cea5932643d260
2,Hotel Bernini Palace,Hotel,"Piazza di San Firenze, 29",43.769440,11.257540,"[{'label': 'display', 'lat': 43.76944, 'lng': ...",165,50100,IT,Firenze,Toscana,Italia,"[Piazza di San Firenze, 29 (Borgo de' Greci), ...",Borgo de' Greci,4bc8848fdc55eee14a85e8ac
3,Il Gattopardo Hotel,Hotel,NaN,43.770559,11.256459,"[{'label': 'display', 'lat': 43.770559, 'lng':...",104,50122,IT,Firenze,Toscana,Italia,"[50122 Firenze Toscana, Italia]",NaN,5cd89186db1d81002cb6e9d8
4,Hotel Calzaiuoli,Hotel,NaN,43.771324,11.255368,"[{'label': 'display', 'lat': 43.77132414522816...",162,NaN,IT,NaN,NaN,Italia,[Italia],NaN,4d95ac749079b1f77604e609


Now some column will be eliminated, mantaining only name, category, address, latitude and longitude.

In [30]:
df_hotel_filtered = df_hotel_filtered[['name','categories', 'address','lat','lng']]
df_hotel_filtered

,name,categories,address,lat,lng
0,Hotel Savoy Firenze,Hotel,Piazza Della Repubblica 7,43.771884,11.254516
1,Relais Hotel Uffizi,Hotel,Via del buco 16,43.769115,11.254923
2,Hotel Bernini Palace,Hotel,"Piazza di San Firenze, 29",43.769440,11.257540
3,Il Gattopardo Hotel,Hotel,NaN,43.770559,11.256459
4,Hotel Calzaiuoli,Hotel,NaN,43.771324,11.255368
5,Hotel Continentale,Hotel,"Vicolo dell'Oro, 6R",43.768586,11.253390
6,HOTEL FIORINO,Hotel,via Osteria del Guanto,43.768079,11.257313
7,Hotel Costantini,Hotel,Via dei Calzaiuoli 13,43.772342,11.255144
8,Hotel Milu ****,Hotel,Via de' Tornabuoni 8,43.770876,11.251488
9,Hotel Lungarno,Hotel,Borgo San Jacopo 14,43.768222,11.251727


Let's visualize the hoels on the map

In [44]:
venues_map_hotel = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred in Florence

# the blue circles represent Florence hotels
for lat, lng, label in zip(df_hotel_filtered.lat, df_hotel_filtered.lng, df_hotel_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_hotel)

# display map
venues_map_hotel